# Computational Mathematics
## An Introduction to Numerical Analysis and Scientific Computing with Python
### By Dimitrios Mitsotakis

# Chapter 12: Eigenvalue Problems

Here we introduce a few methods for the approximation of eigenvalues (and eigenvectors) of matrices

## Power Method

The power method finds the largest in magnitude eigenvalue of a matrix ${\bf A}$ and works only if the maximum in magnitude eigenvalue is simple: 
$|\lambda_1|>|\lambda_2|\geq \cdots \geq |\lambda_N|$

The corresponding eigenvector of the power method can be approximated by the iteration
$${\bf x}^{(k+1)}={\bf A}{\bf x}^{(k)}$$
while the eigenvalue is 
$\lambda_1\approx x^{(k+1)}_j/x^{(k)}_j$ where $j=argmax({\bf x}^{(k)})$.

The Power method can be expressed with the following algorithm:

- Set a tolerance $TOL$ for the accuracy
- Set the maximum number of iterations $MAXIT$
- Initialize the vector ${\bf x}^{(k)}\not={\bf 0}$ and the eigenvalue $\lambda_1=0$, and set $k=0$
- Set $Error = TOL+1$
- Compute $j=argmax_j(x^{(k)}_j)$
- While $Error>TOL$ and $k<MAXIT$
    - Compute new ${\bf x}^{(k+1)}={\bf A} {\bf x}^{(k)}$
    - Set $\lambda_0=x^{(k+1)}_j/x^{(k)}_j$
    - Set $Error = |\lambda_1-\lambda_0|$
    - Increase the counter $k=k+1$
    - Set $\lambda_1=\lambda_0$
- EndWhile
- Return the approximate eigenvalue $\lambda_1$ and the eigenvector
${\bf u}_1=({\bf x}^{(k)}/\lambda_1^k)/\|{\bf x}^{(k)}/\lambda_1^k\|$

and its implementation is the following

In [8]:
def powerMethod(A, x0, tol = 1.e-6, maxit = 100):
    # initialize the parameters
    iteration = 0
    error = tol+1.0
    lambda0 = 0.0
    # find the index for the maximum entry
    j = np.argmax(x0)
    while (error > tol and iteration < maxit):
        x1 = np.dot(A,x0)
        # find the approximation of the eigenvalue
        lambda1 = x1[j]/x0[j]
        error = np.abs(lambda0-lambda1)
        iteration = iteration + 1
        lambda0 = lambda1
        x0 = x1.copy()
    k = iteration
    u1 = (x1/lambda1**k)/npl.norm(x1/lambda1**k,np.inf)
    print('number of iterations required = ',iteration)
    # return eigenvalue and corresponding eigenvector
    return lambda1, u1

As testing example we consider the matrix
$$
{\bf A}=\begin{pmatrix}
-2 & -4 & 2\\
-2 & 1 & 2\\
4 & 2 & 5
\end{pmatrix}
$$
with spectrum $\sigma({\bf A})=\{-5,3,6\}$

In [7]:
import numpy as np
import numpy.linalg as npl

# define matrix and initial guess x0
A = np.array([[-2.,-4.,2.],[-2.,1.,2.],[4.,2.,5.]])
x0 = np.array([1,1,1])
# compute eigenvalue and eigenvector
lambda1,u1 = powerMethod(A,x0, tol=1.e-10, maxit=1000)
# print eigenvalue and eigenvector
print(lambda1)
print(u1)
# compute and print the residual $\|\bA \bu_1-\lambda_1\bu_1\|$
error =  npl.norm(np.dot(A,u1) - lambda1*u1)
print('actual error = ', error)

number of iterations required =  158
6.0000000000407345
[0.0625 0.375  1.    ]
actual error =  4.3158870502569533e-11


## Inverse Power Method

To find the smallest in magnitude eigenvalue we can use the Power method applied to the matrix ${\bf A}^{-1}$. Then the eigenvalue of ${\bf A}$ with the smallest magnitude will be the $1/\lambda_1$ where $\lambda_1$ the maximum in magnitude eigenvalue of ${\bf A}^{-1}$. Because we cannot compute the inverse of ${\bf A}$ that easily we solve the system
$${\bf A}{\bf x}^{(k+1)}={\bf x}^{(k)}$$ 
to compute the corresponding eigenvector.

This technique is known to as the inverse power method.

The inverse power method can be implemented as follows:

In [11]:
import scipy.linalg as spl
def inversePowerMethod(A, x0, tol = 1.e-6, maxit = 100):
    # initialize the parameters
    iteration = 0
    error = tol+1.0
    lambda0 = 0.0
    lu, piv = spl.lu_factor(A)
    # find the index for the maximum entry
    j = np.argmax(x0)
    while (error > tol and iteration < maxit):
        x1 = spl.lu_solve((lu, piv), x0)
        # find the approximation of the eigenvalue
        lambda1 = x1[j]/x0[j]
        error = np.abs(lambda0-lambda1)
        iteration = iteration + 1
        lambda0 = lambda1
        x0 = x1.copy()
    k = iteration
    u1 = (x1/lambda1**k)/npl.norm(x1/lambda1**k,np.inf)
    print('number of iterations required = ',iteration)
    # return eigenvalue and corresponding eigenvector
    return 1/lambda1, u1 

Then the minimum in magnitude eigenvalue of the previous example can be approximated using the following code

In [12]:
A = np.array([[-2.,-4.,2.],[-2.,1.,2.],[4.,2.,5.]])
x0 = np.array([-1,1,1])
lambda1,u1 = inversePowerMethod(A,x0, tol=1.e-10, maxit=1000)
print(lambda1)

number of iterations required =  44
3.0000000002520206


Other eigenvalues can be computed using the shifted inverse power method. But instead we consider the $QR$ iteration, which can approximate all the eigenvalues at once.

## Basic $QR$ Iteration

The basic $QR$ iteration starts with a matrix 
$${\bf T}^{(0)}={\bf A}$$
Then for $k=0,1,\dots$
we compute its $QR$ factorization 
$${\bf Q}^{(k)}{\bf R}^{(k)}=T^{(k)}$$
and we compute the matrix
$$T^{(k+1)}={\bf R}^{(k+1)}{\bf Q}^{(k+1)}$$

The matrix ${\bf T}^{(k)}$ tends to become diagonal with the eigenvalues of ${\bf A}$ in its main diagonal.

The implementation of the basic $QR$ iteration can be as follows:

In [13]:
def basicQRiteration(A, tol = 1.e-6, maxit = 100):
    # initialize the parameters
    iteration = 0
    error = tol+1.0
    T = A.copy()
    #set first approximation of eigenvalues
    eigs = np.diag(T)
    while (error > tol and iteration < maxit):
        #extract the diagonal entries of T
        eig0=eigs
        Q, R = npl.qr(T)
        T = np.dot(R,Q)
        #extract the new diagonal entries
        eigs = np.diag(T)
        error = npl.norm(eigs-eig0)
        iteration = iteration + 1
    print('number of iterations required = ',iteration)
    # return eigenvalue and corresponding eigenvector
    return eigs

Testing this function to our previous example we find

In [14]:
A = np.array([[-2.,-4.,2.],[-2.,1.,2.],[4.,2.,5.]])
eigs = basicQRiteration(A, tol = 1.e-6, maxit = 100)
print(eigs)

number of iterations required =  89
[ 5.99999972 -4.99999972  3.        ]


## Application in Computer Networks

Here we consider a set of webpages where the following number of departing links per node
$$
\begin{array}{ccccccccccc}
i & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 \\
d_1 & 3 & 2 & 3 & 2 & 4 & 2 & 2 & 2 & 2 & 3
\end{array}
$$

This leds to the system ${\bf r}={\bf G}{\bf r}$
where 
$${\bf G}=\begin{pmatrix}
0 & 0 & 0 & 0 & 1/4 & 0 & 0 & 0 & 0 & 0\\
1/3 & 0 & 1/3 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 1/2 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
1/3 & 1/2 & 1/3 & 0 & 1/4 & 0 & 0 & 0 & 0 & 0\\
1/3 & 0 & 0 & 1/2 & 0 & 1/2 & 0 & 0 & 1/2 & 0\\
0 & 0 & 0 & 1/2 & 1/4 & 0 & 0 & 1/2 & 0 & 0\\
0 & 0 & 1/3 & 0 & 0 & 1/2 & 0 & 0 & 0 & 1/3\\
0 & 0 & 0 & 0 & 0 & 0 & 1/2 & 0 & 1/2 & 1/3\\
0 & 0 & 0 & 0 & 1/4 & 0 & 0 & 1/2 & 0 & 1/3\\
0 & 0 & 0 & 0 & 0 & 0 & 1/2 & 0 & 0 & 0\\
\end{pmatrix}$$
is the stochastic Google matrix
and ${\bf r}$ the importance vector of each webpage. We compute the importance vector as an principal eigenvector of $\bf G$ as follows:

In [17]:
A = np.array([[0,0,0,0,1/4,0,0,0,0,0],
              [1/3,0,1/3,0,0,0,0,0,0,0],
              [0,1/2,0,0,0,0,0,0,0,0],
              [1/3,1/2,1/3,0,1/4,0,0,0,0,0],
              [1/3,0,0,1/2,0,1/2,0,0,1/2,0],
              [0,0,0,1/2,1/4,0,0,1/2,0,0],
              [0,0,1/3,0,0,1/2,0,0,0,1/3],
              [0,0,0,0,0,0,1/2,0,1/2,1/3],
              [0,0,0,0,1/4,0,0,1/2,0,1/3],
              [0,0,0,0,0,0,1/2,0,0,0]] )
x0 = np.ones(10)/10.0
lambda1,u1 = powerMethod(A,x0, tol=1.e-10, maxit=1000)
r1 = u1/npl.norm(u1,1)
print(r1)

number of iterations required =  52
[0.05417957 0.02167183 0.01083591 0.08668731 0.21671827 0.16821465
 0.10526316 0.14138287 0.14241486 0.05263158]
